![navis logo](https://github.com/navis-org/navis/raw/master/docs/_static/logo_new_banner.png)

# NeuroPython Workshop 2024 - <span style="color:rgb(250,175,3)">NAVis</span> Exercise 2

_Author: Philipp Schlegel (University of Cambridge)_

_Date: September 23, 2024_

_Source: https://github.com/navis-org/neuropython2024/exercises/navis/navis_exercise_1.ipynb_

In this second exercise we will analyse morphological data from ["Integrated Morphoelectric and Transcriptomic Classification of Cortical GABAergic Cells"](https://www.cell.com/cell/pdf/S0092-8674(20)31254-X.pdf) by Gouwens, Sorensen _et al._ (2020).

You will learn to:

1. Load neurons from disk and attach meta data 
2. Correct offsets
3. Generate publication-ready plots 
4. Extract basic neuron properties 
5. Run a clustering

### Requirements 

As with the previous exercise, you will need a full, up-to-date install of <span style="color:rgb(250,175,3)">NAVis</span>:

```shell 
pip install "navis[all]" -U
```

In addition, you will need the folder with the SWC files and the `20200711_patchseq_metadata_mouse.csv` with the meta data.
Please see the [Data](https://navis-org.github.io/neuropython2024/preparing/#data) section on the website for details!

In [ ]:
import navis

# This should be 1.8.0 or higher
print(navis.__version__)

### Part I: Loading neurons

Importing skeletons from disk is pretty straight forward using `navis.read_swc`:

In [ ]:
# Read the SWC files in the folder
nl = navis.read_swc(
    "~/Downloads/swc/",  # adjust the filepath as needed!
    fmt="{name,id:int}_transformed.swc",  # parse the name and id from the file name
    limit=".*_transformed.swc",  # only read files that end with `_transformed.swc`
)
nl

Next, we want to load the meta data provided alongside the reconstructions:

In [ ]:
import pandas as pd

meta = pd.read_csv("~/Downloads/20200711_patchseq_metadata_mouse.csv")

# Strangely, the last column with the transcriptomic ("t") types has no name - let's fix that:
meta.columns = list(meta.columns[:-1]) + ["t_type"]

meta.head()

The `cell_specimen_id` should match our neurons' `.id` property. The columns we care about are:

- `cell_soma_normalized_depth` to help us position the neurons along the y-axis for plotting
- `neuron_reconstruction_type` to let us drop incomplete reconstructions 
- `t_type` to let us correlate our morphological analyses with the transcriptomic types

In [ ]:
# Attach metadata to neurons in the list
nl.add_metadata(
    meta,
    id_col="cell_specimen_id",  # the column in the metadata that corresponds to the neuron id
    columns=[
        "t_type",
        "neuron_reconstruction_type",
        "cell_soma_normalized_depth",
    ],  # only get our subset of columns
    register=True,  # use this meta data for the summary table
)

nl

In [ ]:
# With the meta data attached we can do stuff like this:
nl[nl.t_type == "Lamp5 Ntn1 Npy2r"]

Next, we need to align the neurons according to their soma depth! The normalized `cell_soma_normalized_depth` should map to a physical range of `0` to `922.5861720311`.

Let's demo with one neuron before we run this for all neurons:

In [ ]:
# Grab one of the neurons
n = nl[0]

# This is the normalized soma depth
print(f"Normalized soma depth: {n.cell_soma_normalized_depth}")

# This is the physical soma depth
# Note that we're positioning from the bottom, i.e. 922 will be the surface of the slice
# That's because matplotlib uses the bottom left as origin, so we need the "deepest" neurons
# to be at y = 0
phys_y = (1 - n.cell_soma_normalized_depth) * 922.5861720311
print(f"Physical soma depth: {phys_y}")

# Current soma
print(f"Current soma coordinates: {n.soma_pos[0]}")

We will now offset the neuron such that the soma is at x=0 and y=368.6:

In [ ]:
offset = [0, phys_y, 0] - n.soma_pos[0]
offset

Moving or scaling neurons in <span style="color:rgb(250,175,3)">NAVis</span> is super straight forward: adding, subtracting, dividing or multiplying neurons by a number or an `[x, y, z]` vector will change their coordinates. 

In [ ]:
# Move the neuron to the new centered position
n += offset

# Check the that the soma is in the correct position
n.soma_pos[0]

That looks good! Let's do it for all neurons:

In [9]:
for n in nl:
    phys_y = (1 - n.cell_soma_normalized_depth) * 922.5861720311
    offset = [0, phys_y, 0] - n.soma_pos[0]
    n += offset

In [ ]:
# Check that soma positions are correct
nl.soma_pos

## Part II: Plotting

Now that we have loaded and aligned the neurons, let's see if we can recreate a plot similar to those in Figure 4A:

![Figure4A](_static/Fig4_A.png)

Here is the complete code for plotting in a single function for re-use. During the workshop we will build this slowly bit by bit to make it easier to understand!

In [ ]:
def plot_t_type(t_type, color="magenta", axon_color="purple", offset=500):
    """Plot all neurons of a given transcriptomic type.

    Parameters
    ----------
    t_type : str
        The transcriptomic type to plot.
    color : str
        The color of the dendrites.
    axon_color : str
        The color of the axon.
    offset : int
        The offset between neurons along the x-axis.

    Returns
    -------
    fig, ax
        The matplotlib figure and axis.

    """
    to_plot = nl[nl.t_type == "Lamp5 Ntn1 Npy2r"]

    # Offset the neurons slightly along the x-axis
    to_plot = [n + [offset * i, 0, 0] for i, n in enumerate(to_plot)]

    compartment_palette = {i: color for i in range(5)}
    compartment_palette[2] = axon_color

    # Plot the neuron
    fig, ax = navis.plot2d(
        to_plot,
        radius=False,  # use thin lines
        lw=1,
        c="black",
        method="2d",
        soma=dict(
            fc="black",
            ec="white",  # highlight the soma with a white outline
            radius=10,  # override the default soma radius
        ),
        color_by="label",
        figsize=(
            len(to_plot) * 2,
            10,
        ),  # scale the figure size with the number of neurons
        palette=compartment_palette,
    )

    # Add the layer boundaries
    layer_bounds = {
        "L1": 0,
        "L2/3": 115.1112491335,
        "L4": 333.4658190171,
        "L5": 453.6227158132,
        "L6": 687.6482650269,
        "L6b": 883.1308910545,
    }

    for layer, y in layer_bounds.items():
        y = 922.5861720311 - y
        ax.axhline(y, color="gray", ls="--", lw=1)
        ax.text(-300, y - 25, layer, color="gray", va="center", size=11)

    ax.axhline(0, color="gray", ls="--", lw=1)

    # Set the axis y limits according to the layers
    ax.set_ylim(-10, 930)

    # Hide axis
    ax.axis("off")

    return fig, ax


fig, ax = plot_t_type("Lamp5 Ntn1 Npy2r")

Next, let's calculate the distribution of cable lengths:

In [ ]:
# Take one of the Lamp5 Ntn1 Npy2r neurons
to_plot = nl[nl.t_type == "Lamp5 Ntn1 Npy2r"]

# We're going to be cheap here and simply generate a histogram over the node positions
# To make this representative, we should make sure that the number of nodes per unit of cable
# is homogeneous across neurons. For that we will resample the neurons:
print(
    f"Sampling rate (nodes/cable) before resampling: {to_plot.sampling_resolution.mean():.2f}"
)

# Resample to 2 nodes per micron
to_analyze = navis.resample_skeleton(
    to_plot,
    resample_to=0.5,
    map_columns="label",  # make sure label column is carried over
)
print(
    f"Sampling rate (nodes/cable) after resampling: {to_analyze.sampling_resolution.mean():.2f}"
)

In [ ]:
# Get the combined nodes table
nodes = to_analyze.nodes
nodes.head()

In [ ]:
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Plot the neurons using the function we defined earlier
fig, ax = plot_t_type("Lamp5 Ntn1 Npy2r")

# Add a new axis to the right of the main plot
divider = make_axes_locatable(ax)
ax_hist = divider.append_axes("right", size=0.75, pad=0.05)

# Add histograms
# For axon:
sns.kdeplot(
    data=nodes[nodes.label == 2], y="y", ax=ax_hist, color="purple", linewidth=1.5
)
# For the rest:
sns.kdeplot(
    data=nodes[nodes.label != 2], y="y", ax=ax_hist, color="magenta", linewidth=1.5
)

# Add soma positions
soma_pos = to_plot.soma_pos.reshape(-1, 3)
ax_hist.scatter([0] * len(soma_pos), soma_pos[:, 1], color="black", s=10, clip_on=False)

# Set same axis limits as the main plot
ax_hist.set_ylim(-10, 930)

# Hide axes
ax_hist.set_axis_off()

## Part III: Morphometrics

For the last part of this exercise, we will extract a number of features from each neuron and try to run a simple clustering. The idea is to create something like Figure 5A to compare transcriptomic with morphology types:

![Fig5a](_static/Fig5_A.jpg)

For their morphology types, Gouwens, Sorensen _et al._ used a collection of features collectively called "IVSCC" (In-Vitro Single Cell Characterization). The exact set of features seems to vary between studies but here are some examples:

- Axon-soma distance: the path distance from the axon root to the soma surface (μm)
- Total length: the combined length of all branches (μm)
- Max. path distance: the path distance from the soma to the furthest neurite tip (μm)

You can find a short description for each feature in the Supplements Note for ["Classification of electrophysiological and
morphological neuron types in the mouse visual cortex"](https://www.nature.com/articles/s41593-019-0417-0) by Gouwens, Sorensen, Berg _et al._ (2019).

We will start by calculating a few of these by hand before switching of to letting <span style="color:rgb(250,175,3)">NAVis</span> do the heavy lifting!

First item on the menu: total length!

In [ ]:
# Grab one example neuron to analyze
n = nl[0]

print(f"Total length: {n.cable_length:.1f} microns")

Ok, that was easy. But what if you wanted to get the path length for axon and dendrites separately?

For that, we need to split the neuron using `navis.subset_neuron`:

In [16]:
n_axon = navis.subset_neuron(n, subset=n.nodes.label == 2)  # axon only
n_dend = navis.subset_neuron(n, subset=n.nodes.label != 2)  # everything but the axon

In [ ]:
print(f"Axon length: {n_axon.cable_length:.1f} microns")
print(f"Dendrites length: {n_dend.cable_length:.1f} microns")

Great! How about something a bit more complicated next? How about max path distance, i.e. the longest distance between any two points in the neuron?

In the paper, they calculate both the max Euclidean and the max geodesic (along-the-arbor) distance:

In [ ]:
from scipy.spatial.distance import pdist

# Get the maximum pairwise distance between nodes
max_distance = pdist(n.nodes[["x", "y", "z"]]).max()

print(f"Maximum Euclidean path distance: {max_distance:.1f} microns")

In [ ]:
# Calculate the geodesic distance between leaf nodes and all other nodes
m = navis.geodesic_matrix(n, from_=n.leafs.node_id.values)

# Rows are our leaf nodes; columns are all other nodes
m.head()

In [ ]:
# The maximum geodesic distance
max_distance = m.values.max()

print(f"Maximum geodesic path distance: {max_distance:.1f} microns")

One important note regarding the `geodesic_matrix` above: if you neuron consists of multiple disconnected pieces, some of the distances will be infinite (`np.inf`). If that's the case, you need to either reconnect the neuron (`navis.heal_skeleton`) or exclude infinite values (`m.values[m.values < np.inf].max()`)!

Let's do something slightly more complicated for the last example feature:

- Mean contraction: the average of the ratios of the summed Euclidean distance between bifurcations, and between bifurcations and tips, to the summed path distance between same

That explanation may sound a bit complicated but it's actually fairly simple: it is asking, for each linear segment between branch/leaf nodes, what's the ratio of the direct Euclidean vs the along-the-arbor geodesic distance:

![skeleton](_static/tortuosity.jpg)

Let's try calculating this manually, shall we?

In [ ]:
# NAVis pre-computes two kinds of segments:
print("Linear segmnents that maximmize segment lengths:", len(n.segments))
print("Linear segmnents between branch/leaf nodes:", len(n.small_segments))

# For the tortuosity we need to use the latter!

# Each segment is a list of node IDs from distal to proximal
n.small_segments[0]

Let's first calculate the Euclidean distance:

In [ ]:
import numpy as np

# Calculate the Euclidean distance between the first and the last node of each segment
node_locs = dict(zip(n.nodes.node_id, n.nodes[["x", "y", "z"]].values))
dist_eucl = []
for seg in n.small_segments:
    dist_eucl.append(np.linalg.norm(node_locs[seg[0]] - node_locs[seg[-1]]))
dist_eucl = np.array(dist_eucl)

# Inspect the results
dist_eucl[:10]

Next, we need the geodesic distance. Here you have multiple options! The fastest is probably using `navis.geodesic_matrix` again but since we've already demo'ed that, let's run this operation on the neuron's graph representation instead:

In [ ]:
# NAVis precomputed both iGraph and networkx graphs for the neuron
n.graph, n.igraph

iGraph is faster but we have to contend with mapping between node indices and vertex IDs which is annoying. Let's use networkx instead:

In [ ]:
import networkx as nx

dist_geo = []
for seg in n.small_segments:
    dist_geo.append(
        nx.shortest_path_length(n.graph, source=seg[0], target=seg[-1], weight="weight")
    )
dist_geo = np.array(dist_geo)
dist_geo[:10]

Great! The only thing that's left is to calculate the ratios and take the mean:

In [ ]:
tort = (dist_geo / dist_eucl).mean()
print(f"Tortuosity: {tort:.3f}")

For the record: <span style="color:rgb(250,175,3)">NAVis</span> has a built-in function to calculate tortuosity:

In [ ]:
navis.tortuosity(n)

We hope that manually compiling a couple features may have given you an idea of how this is done in general. In particular the use of the graph representation (`.graph` or `.igraph`) can be very powerful. We also encourage you to search the [API documentation](https://navis-org.github.io/navis/api/) to see if there already is a function for what you have in mind.

Moving on, let's use <span style="color:rgb(250,175,3)">NAVis</span> to collect our IVSCC features:

In [ ]:
ivscc = navis.ivscc_features(n)
ivscc

The `ivscc_features` function is new in <span style="color:rgb(250,175,3)">NAVis</span> `1.8.0` and currently calculates up to 57 features depending on which neuron compartments are present.

Let's run this for all neurons in our list:

In [ ]:
ivscc = navis.ivscc_features(nl)
ivscc.head()

In [29]:
# Manually add the normalized soma depth to the ivscc table
ivscc.loc["cell_soma_normalized_depth"] = nl.cell_soma_normalized_depth

Now that we have the features, we can run a clustering on them. There are obviously tons of options for doing that including many ways to pre-process the data.  Like in the paper, we will run a hierarchical clustering:

In [ ]:
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, cut_tree

# Drop neurons that don't have a t-type assigned
nl_w_ttype = nl[nl.t_type != "nan"]

# First we will drop neurons that have not t-type assigned
ivscc_w_ttype = ivscc[nl_w_ttype.id]

# Our IVSCC features contain some NaNs where neurons don't have a basal or apical dendrite
# We could throw them out but let's just fill them with zeros
ivscc_filled = ivscc_w_ttype.fillna(0)

# Normalize the data to be between 0 and 1
ivscc_scaled = (ivscc_filled - ivscc_filled.min(axis=1).values.reshape(-1, 1)) / (
    ivscc_filled.max(axis=1) - ivscc_filled.min(axis=1)
).values.reshape(-1, 1)

ivscc_scaled.fillna(0, inplace=True)

# Calculate the Euclidean distance
dist = pdist(ivscc_scaled.T)

# Perform hierarchical clustering
Z = linkage(dist, method="ward")

# Cut the tree to get the clusters
labels = cut_tree(Z, n_clusters=40).flatten()

# Combine cluster labels and t-types into a dataframe
clusters = pd.DataFrame()

clusters["id"] = nl_w_ttype.id
clusters["t_type"] = nl_w_ttype.t_type
clusters["cluster"] = labels
clusters.head()

In [ ]:
from scipy.cluster.hierarchy import leaves_list

# For plotting we need to create a confusion matrix between the clusters and the t-types
confusion = pd.crosstab(clusters.t_type, clusters.cluster)

# Sort rows and columns by hierarchical clustering
row_linkage = linkage(pdist(confusion), method="ward")
row_order = leaves_list(row_linkage)
col_linkage = linkage(pdist(confusion.T), method="ward")
col_order = leaves_list(col_linkage)

# Apply the sorting
confusion = confusion.iloc[row_order, col_order]

confusion.head()

In [32]:
# For each t-type, generate a color based on how the first label
first_labels = np.unique(confusion.index.map(lambda x: x.split(" ")[0]))

cmap = {}
for t, base_color in zip(first_labels, sns.color_palette("tab10", len(first_labels))):
    # All labels starting with `t`
    t_labels = [ix for ix in confusion.index if ix.startswith(t)]
    cmap.update(
        dict(
            zip(
                t_labels,
                sns.light_palette(base_color, n_colors=len(t_labels) * 2)[::-1],
            )
        )
    )

In [ ]:
# Plot the confusion matrix as scatter heatmap
confusion_pivot = confusion.unstack().reset_index().rename(columns={0: "count"})


# Turn the cluster into a string - otherwise seaborn will re-order it
confusion_pivot = confusion_pivot.astype({"cluster": "string", "t_type": "string"})

g = sns.relplot(
    data=confusion_pivot,
    y="t_type",
    x="cluster",
    size="count",
    palette=cmap,
    hue="t_type",
    edgecolor=".7",
    col_order=confusion.index.values,
    row_order=confusion.columns.values,
    height=10,
    sizes=(0, 400),
    legend=False,
)

ax = g.axes[0, 0]

ax.set_ylabel("Transcriptomic type")
ax.set_xlabel("Morphological cluster")

That looks very different from the figure in the paper. However, in the paper they:

- used more features
- removed low-variance features
- ran multiple clusterings and extracted consensus clusters
- merged small clusters (n<3) into the next larger cluster
- etc.

Bottom line: it's not suprising we don't get the same results off the bat. Still, we hope this has given you an idea of how to run clustering on morphological features! Also check out the CAJAL tutorial for a very different approach on calculating morphological similarity!